In [1]:
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = False
from matplotlib import cm, colors
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import pandas as pd
from astropy.table import Table, unique
from astropy.coordinates import SkyCoord, match_coordinates_sky

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import pandas as pd
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
from astropy.table import join
from scipy.optimize import minimize, curve_fit

In [2]:
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.table import Table, vstack, hstack, join, unique
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

from datetime import datetime, timedelta

from scipy.ndimage import gaussian_filter1d

import os
from glob import glob

import psycopg2

from tqdm.notebook import tqdm_notebook

In [3]:
from hyperfit.linfit import LinFit

ModuleNotFoundError: No module named 'hyperfit'

In [ ]:
h = 1
H0 = 100*h

c = 3e5

q0 = 0.2

In [ ]:
def firstdigit(n):
    """Return the first digit of a number.
    
    Parameters
    ----------
    n : int, float, or ndarray
        Number or list of numbers.
    
    Returns
    -------
    digit : int
        First digit of the number.
    """
    return np.trunc(n * 10**(-np.trunc(np.log10(n)))).astype(int)

def plot_radec(ra, dec):
    """Mollweide projection plot adapted to astro coordinates.
    
    Parameters
    ----------
    ra : pandas.Series or list
        List of candidate RA [deg].
    dec : pandas.Series or list
        List of candidate Dec [deg].
    
    Returns
    -------
    fig : matplotlib.Figure
        Figure object to let user apply further plot manipulation.
    """
    # Convert RA, Dec to radians.
    # Rotate the RA so that the plot goes 360->0 left to right.
    _ra = np.radians(120 - ra)
    _ra[_ra < -np.pi] += 2*np.pi
    _dec = np.radians(dec)

    fig, ax = plt.subplots(1,1, figsize=(8,4), subplot_kw={'projection': 'mollweide'})
    ax.scatter(_ra, _dec, alpha=0.5, s=2)
    ax.set(xticks=np.radians([-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150]),
           xticklabels=['270', '240', '210', '180', '150', '120', '90', '60', '30', '0', '330'])
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.grid(ls=':')
    fig.tight_layout()
    
    return fig;

import requests

def get_cutout(targetid, ra, dec, width=300, height=300, verbose=False):
    """Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    targetid : int
        DESI target ID.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    w : astropy.wcs.WCS
        World coordinate system for the image.
    """
    # Either load an existing image or download a cutout.
    os.makedirs('_cache', exist_ok=True)
    img_name = f'_cache/cutout_{targetid}_{width}_{height}.jpg'
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))
    else:
        if verbose:
            print('Accessing {}'.format(img_name))
        img_url = f'https://www.legacysurvey.org/viewer/cutout.jpg?ra={ra}&dec={dec}&%22/pix=0.25&layer=ls-dr9&width={width}&height={height}&sga'
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
                
    # Set up the WCS.
    wcs_input_dict = {
        'CTYPE1': 'RA---TAN',
        'CUNIT1': 'deg',
        'CDELT1': -0.25/3600,
        'CRPIX1': width//2,
        'CRVAL1': ra,
        'NAXIS1': width,
        'CTYPE2': 'DEC--TAN',
        'CUNIT2': 'deg',
        'CDELT2': 0.25/3600,
        'CRPIX2': height//2,
        'CRVAL2': dec,
        'NAXIS2': height
    }
    w = WCS(wcs_input_dict)
    
    return img_name, w

In [ ]:
def clean_catalog(data):
    """Take a catalog matched to PV TF targets and apply quality cuts:
    * Keep only good redshifts (ZWARN==0, DELTACHI2>=25)
    * Ensure 1 main DESI TARGETID per SGA_ID
    * Ensure >1 distinct TARGETIDs per SGA_ID
    * Ensure targets are not all spatially coincident.
    
    Parameters
    ----------
    data : astropy.Table
        Table of TF redshift measurements and matched SGA_IDs.
    
    Returns
    -------
    data_tf: astropy.Table or None
        Data suitable for Tully-Fisher analysis, after basic cuts.
    """
    # All targets
    _ids_all, _counts_all = np.unique(tfuji['SGA_ID'], return_counts=True)

    # Identify targets with good redshifts.
    isgoodz = (data['ZWARN']==0) & (data['DELTACHI2']>=25)
    _ids_goodz, _counts_goodz = np.unique(data['SGA_ID'][isgoodz], return_counts=True)

    # Select SGA_IDs with at least 2 good associated redshifts.
    select = np.in1d(data['SGA_ID'], _ids_goodz[_counts_goodz > 1]) & isgoodz
    data = data[select].group_by('SGA_ID')

    # Storage for output.
    data_tf = None

    # Loop through the table and keep only SGA IDs with >= 2 unique TARGETIDs, where one is a MAIN survey target.
    sga_ids = np.unique(data['SGA_ID'])
    N = len(sga_ids)

    with tqdm_notebook(total=N) as progress_bar:
        for i, sga_id in enumerate(sga_ids):
            progress_bar.update(1)
            tab = data[data['SGA_ID']==sga_id]

            # MAIN targets have a TARGETID that starts with 3. Ensure one is present.
            digits = firstdigit(tab['TARGETID'])
            if np.any(digits == 3):
                maintargetids = np.unique(tab['TARGETID'][digits==3].value)

                # Ensure there are at least two distinct TARGETIDs matched to this SGA_ID.
                ntargets = len(np.unique(tab['TARGETID'].value))
                if ntargets >= 2:

                    # Ensure the TARGETIDs correspond to distinct locations on the sky.
                    coords = SkyCoord(tab['TARGET_RA'], tab['TARGET_DEC'], frame='icrs', unit='degree')
                    is_distinct = np.any([c1.separation(c2).to_value('arcsec') > 1 for c1 in coords for c2 in coords])
                    if not is_distinct:
                        continue
                # Check that there two or more TARGETIDs.
                else:
                    continue
            # Check that there is a MAIN survey TARGETID associated with this SGA_ID.
            else:
                continue

            if data_tf is None:
                data_tf = tab
            else:
                data_tf = vstack([data_tf, tab])

    return data_tf

In [ ]:
def clean_catalog_1(data):
    """Take a catalog matched to PV TF targets and apply quality cuts:
    * Keep only good redshifts (ZWARN==0, DELTACHI2>=25)
    * Ensure 1 main DESI TARGETID per SGA_ID
    * Ensure >1 distinct TARGETIDs per SGA_ID
    * Ensure targets are not all spatially coincident.
    
    Parameters
    ----------
    data : astropy.Table
        Table of TF redshift measurements and matched SGA_IDs.
    
    Returns
    -------
    data_tf: astropy.Table or None
        Data suitable for Tully-Fisher analysis, after basic cuts.
    """
    # All targets
    _ids_all, _counts_all = np.unique(tfuji['SGA_ID'], return_counts=True)

    # Identify targets with good redshifts.
    isgoodz = (data['ZWARN']==0) & (data['DELTACHI2']>=25)
    _ids_goodz, _counts_goodz = np.unique(data['SGA_ID_1'][isgoodz], return_counts=True)

    # Select SGA_IDs with at least 2 good associated redshifts.
    select = np.in1d(data['SGA_ID_1'], _ids_goodz[_counts_goodz > 1]) & isgoodz
    data = data[select].group_by('SGA_ID_1')

    # Storage for output.
    data_tf = None

    # Loop through the table and keep only SGA IDs with >= 2 unique TARGETIDs, where one is a MAIN survey target.
    sga_ids = np.unique(data['SGA_ID_1'])
    N = len(sga_ids)

    with tqdm_notebook(total=N) as progress_bar:
        for i, sga_id in enumerate(sga_ids):
            progress_bar.update(1)
            tab = data[data['SGA_ID_1']==sga_id]

            # MAIN targets have a TARGETID that starts with 3. Ensure one is present.
            digits = firstdigit(tab['TARGETID'])
            if np.any(digits == 3):
                maintargetids = np.unique(tab['TARGETID'][digits==3].value)

                # Ensure there are at least two distinct TARGETIDs matched to this SGA_ID.
                ntargets = len(np.unique(tab['TARGETID'].value))
                if ntargets >= 2:

                    # Ensure the TARGETIDs correspond to distinct locations on the sky.
                    coords = SkyCoord(tab['TARGET_RA'], tab['TARGET_DEC'], frame='icrs', unit='degree')
                    is_distinct = np.any([c1.separation(c2).to_value('arcsec') > 1 for c1 in coords for c2 in coords])
                    if not is_distinct:
                        continue
                # Check that there two or more TARGETIDs.
                else:
                    continue
            # Check that there is a MAIN survey TARGETID associated with this SGA_ID.
            else:
                continue

            if data_tf is None:
                data_tf = tab
            else:
                data_tf = vstack([data_tf, tab])

    return data_tf

### Fuji

In [ ]:
tfuji = Table.read('/global/project/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits')
tfuji

### Daily Tiles

In [ ]:
tdaily = Table.read('/global/project/projectdirs/desi/science/td/pv/desi_pv_tf_daily_tiles.fits')
tdaily = unique(tdaily[(tdaily['NIGHT'] > 20210513) & (tdaily['NIGHT'] < 20220514)])
tdaily

#### Multiple Counts

In [ ]:
# All targets
_ids_all, _counts_all = np.unique(tdaily['SGA_ID'], return_counts=True)
_ids_all[_counts_all > 1]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,5), tight_layout=True)

ax.hist(_counts_all[_counts_all > 1], bins = 20)

ax.set(xlabel='Observations per SGA_ID in Daily Tiles', 
       ylabel='count');

In [ ]:
# All targets with decent redshifts and observations suitable for the Tully-Fisher analysis
tmain_tf = clean_catalog(tdaily)
_ids_goodz, _counts_goodz = np.unique(tmain_tf['SGA_ID'], return_counts=True)
_ids_goodz[_counts_goodz > 1]

In [ ]:
tmain_goodz = np.in1d(tmain_tf['SGA_ID'], _ids_goodz[_counts_goodz > 1])
tmain_g = tmain_tf[tmain_goodz]
tmain_g

#### Inclination Cut

In [ ]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

SGA[:10]

In [ ]:
SGA_main = join(tmain_g, SGA, keys_left='SGA_ID', keys_right='SGA_ID')

In [ ]:
SGA_main.group_by('SGA_ID_1')

In [ ]:
sga_ids_clean = []

rmag_clean = []
drmag_clean = []
vmax_clean = []
dvmax_clean = []
z = []

q0 = 0.2

inc_min = 45*u.degree
cosi_max = np.cos(inc_min.to('radian'))

SGA_main['cosi'] = np.sqrt((SGA_main['BA']**2 - q0**2)/(1 - q0**2))
SGA_main['cosi'][np.isnan(SGA_main['BA'])] = 0 # Objects with b/a < 0.2

In [ ]:
for col in SGA_main.colnames:
    SGA_main = SGA_main[SGA_main['cosi'] < cosi_max]

In [ ]:
SGA_main

#### Morphology Cut

for col in SGA_main.colnames:
    SGA_main = SGA_main[SGA_main['MORPHTYPE'].startswith('S')]

In [ ]:
for j in range(0,20): 
    for i in range(0, len(SGA_main)):
        morphtype = str(SGA_main['MORPHTYPE'][i])
    
        # Cut any suspected ellipticals
        if morphtype.startswith('S'):
            continue
        
    #print(i, morphtype)
    
        SGA_main.remove_row(i)

In [ ]:
# All targets
_ids_all, _counts_all = np.unique(SGA_main['SGA_ID_1'], return_counts=True)
_ids_all[_counts_all > 1]

## Clusters

In [ ]:
hdu = fits.open('DESI_SGA/TF/Tully15-Table3.fits')
table3 = Table(hdu[1].data)
hdu.close()

#table3[:100]

In [ ]:
hdu = fits.open('DESI_SGA/TF/Tully13-Table2.fit')
table2 = Table(hdu[1].data)
hdu.close()

#table2[:10]

### Abell 2151

In [ ]:
a2151_nest = 100007

a2151_row_t3 = table3['Nest'] == a2151_nest

R2t_a2151 = table3['R2t'][a2151_row_t3][0]
sigma_a2151 = table3['sigP'][a2151_row_t3][0]

In [ ]:
a2151_coords = SkyCoord(table3['SGLON'][a2151_row_t3]*u.degree, 
                       table3['SGLAT'][a2151_row_t3]*u.degree, 
                       frame='supergalactic')

a2151_group_coords = SkyCoord(table2['SGLON']*u.degree, 
                        table2['SGLAT']*u.degree, 
                        frame='supergalactic')

idx, d2d, d3d = a2151_coords.match_to_catalog_sky(a2151_group_coords)

V_a2151 = table2['__HV_'][idx][0]

In [ ]:
# First, we need to convert R2t from Mpc to an angle, using the group's heliocentric velocity
R2t_a2151_angle = (R2t_a2151/(V_a2151/H0))*u.radian

In [ ]:
a2151_tf_coords = SkyCoord(SGA_main['TARGET_RA'], SGA_main['TARGET_DEC'], unit='deg')

a21sep = a2151_coords.separation(a2151_tf_coords)

In [ ]:
fuji_in_a21511 = (a21sep < 1.5*R2t_a2151_angle) & (SGA_main['Z']*c > V_a2151 - 3*sigma_a2151) & (SGA_main['Z']*c < V_a2151 + 3*sigma_a2151)

fuji_in_a21512 = (a21sep >= 1.5*R2t_a2151_angle) & (a21sep < 3*R2t_a2151_angle) & (SGA_main['Z']*c > V_a2151 - 2*sigma_a2151) & (SGA_main['Z']*c < V_a2151 + 2*sigma_a2151)

fuji_in_a2151 = fuji_in_a21511 | fuji_in_a21512

################################################################################
# Keep all instances of each SGA_ID that are within the Coma cluster
#-------------------------------------------------------------------------------
fuji_ID_in_a2151 = np.unique(SGA_main['SGA_ID_1'][fuji_in_a2151])

idx_fuji_in_a2151 = np.in1d(SGA_main['SGA_ID_1'], fuji_ID_in_a2151)

inAbell2151_fuji_table = SGA_main[idx_fuji_in_a2151]
################################################################################

inAbell2151_fuji_table

In [ ]:
plt.hist(a21sep[fuji_in_a2151].to_value('degree'))
plt.xlabel('MaNGA-Abell 2151 Angular Separation [deg]')
plt.ylabel('Count')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5), tight_layout=True)
ax = axes[0]
ax.plot(inAbell2151_fuji_table['TARGET_DEC'], inAbell2151_fuji_table['TARGET_RA'], '.')
ax = axes[1]
ax.hist(inAbell2151_fuji_table['Z']);

#### Multiple counts

In [ ]:
ids, counts = np.unique(inAbell2151_fuji_table['SGA_ID_1'], return_counts=True)
ids[counts > 1]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,5), tight_layout=True)

ax.hist(counts[counts > 1], bins=np.linspace(2,10,20))

ax.set(xlabel='Observations per SGA_ID Abell 2151', 
       ylabel='count');

In [ ]:
# Keep only those with more than one observation
high_count_sga = np.in1d(inAbell2151_fuji_table['SGA_ID_1'], ids[counts > 1])
tf_a2151_multiple = inAbell2151_fuji_table[high_count_sga]
tf_a2151_multiple

#### Calculate Rotational Velocities

In [ ]:
sga_ids_vel_cuts = []
R26 = []
rmag = []
drmag = []
vmax = []
dvmax = []

for i, sga_id in enumerate(np.unique(inAbell2151_fuji_table['SGA_ID_1'])):
    # if sga_id == 474614:
    #     print('skipped')
    #     continue
    galaxy_list = inAbell2151_fuji_table[inAbell2151_fuji_table['SGA_ID_1'] == sga_id]
    # print
    #print(i+1, sga_id)
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    r26 = 0.5 * float(sga_galaxy['D26'])*u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26
    
    zc, zc_err = sga_galaxy['Z'], sga_galaxy['ZERR']
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = c * dz
    dv_err = c * dz_err
    
    good_vel = dv < 5000
    # print(good_vel[0], sga_id)
    
    if np.sum(good_vel) > 0:
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26])
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2))
            
            R26.append(0.5 * sga_galaxy['D26']*u.arcmin)
            rmag.append(float(sga_galaxy['R_MAG_SB26']))
            drmag.append(float(sga_galaxy['R_MAG_SB26_ERR']))
            vmax.append(v033)
            dvmax.append(dv033)
            sga_ids_vel_cuts.append(sga_id)
    # break
#print('mag:', rmag)
#print('vel:', vmax)
#print('dv:', dvmax)

In [ ]:
rmag = np.asarray(rmag)
drmag = np.asarray(drmag)
vmax = np.asarray(vmax)
dvmax = np.asarray(dvmax)
R26 = np.asarray(R26)

isrmeas = rmag > 0

fig, axes = plt.subplots(1, 2, figsize=(10,4), tight_layout=True)

ax = axes[0]
ax.errorbar(rmag[isrmeas], vmax[isrmeas], xerr=drmag[isrmeas], yerr=dvmax[isrmeas], fmt='ro')
ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylim=(-50,300),
       ylabel='$v_\mathrm{max}$ [km s$^{-1}$]')

ax = axes[1]

log10vmax = np.log10(vmax)
dlog10vmax = 0.434 * dvmax / vmax 

ax.errorbar(rmag[isrmeas], log10vmax[isrmeas], yerr=dlog10vmax[isrmeas], fmt='ro')
ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       ylim=(-0.5,3))

fig.suptitle(r'Rotational velocity at $0.4R_{26}$ for Abell 2151', y=1.05)
fig.subplots_adjust(top=0.8)

print(np.sum(isrmeas))

In [ ]:
sga_ids_clean = []

rmag_clean = []
drmag_clean = []
vmax_clean = []
dvmax_clean = []
z = []

inc_min = 45*u.degree
cosi_max = np.cos(inc_min.to('radian'))

inAbell2151_fuji_table['cosi'] = np.sqrt((inAbell2151_fuji_table['BA']**2 - q0**2)/(1 - q0**2))
inAbell2151_fuji_table['cosi'][np.isnan(inAbell2151_fuji_table['cosi'])] = 0 # Objects with b/a < 0.2

#i = 0

# for sga_id in np.unique(inComa_sga_table['SGA_ID']):
for sga_id in sga_ids_vel_cuts:
    galaxy_list = inAbell2151_fuji_table[inAbell2151_fuji_table['SGA_ID_1'] == sga_id]
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    
    morphtype = str(sga_galaxy['MORPHTYPE'])
    
    cosi = float(sga_galaxy['cosi'])
    
    r26 = 0.5 * float(sga_galaxy['D26']) * u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26

    # Cut any suspected ellipticals
    if morphtype.startswith('E') or morphtype.startswith('S0') or morphtype.startswith('I'):
        continue
               
    # Inclination cut
    if cosi > cosi_max:
        continue
        
    #i += 1
    #print(i, sga_id, cosi)
    
    zc, zc_err = float(sga_galaxy['Z']), float(sga_galaxy['ZERR'])
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = 3e5 * dz
    dv_err = 3e5 * dz_err
    
    good_vel = dv < 5000
    
    if np.sum(good_vel) > 0:
    
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26]) / np.sqrt(1 - cosi**2)
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2)) / np.sqrt(1 - cosi**2)
            z.append(zc)
            rmag_clean.append(float(sga_galaxy['R_MAG_SB26']))
            drmag_clean.append(float(sga_galaxy['R_MAG_SB26_ERR']))
            vmax_clean.append(v033)
            dvmax_clean.append(dv033)
            sga_ids_clean.append(sga_id)
    
#inComa_sga_table[['SGA_ID', 'BA', 'cosi']].show_in_notebook()
print(len(rmag_clean))

In [ ]:
def l1norm(pars, x, y, dy):
    '''
    Linear fit that uses the l1-norm (robust against outliers).
    '''
    a, b = pars
    return np.sum(np.abs((y - a - b*x)/dy))

def l1norm_noerror(pars, x, y):
    '''
    Linear fit that uses the l1-norm without normalizing by measurement uncertainties.
    '''
    a, b = pars
    return np.sum(np.abs(y - a - b*x))

def l2norm(pars, x, y, dy):
    '''
    Linear fit that uses the l2-norm
    '''
    a, b = pars
    return np.sum((y - a - b*x)**2/dy**2)

def fit_tfr(r, logv, dlogv):
    fmin = 1e99
    a, b = 6, -0.25
    hess_inv = np.ones((2,2))
    
    succ_res = None
    
    # Try a large number of random seeds to ensure a decent fit.
    for i in range(1000):
        _a, _b = np.random.uniform(0,10), np.random.uniform(-1,0)
        
        res = minimize(l1norm_noerror, 
                       [_a, _b], 
                       args=(r, logv),# dlogv),
                       method='L-BFGS-B', 
                       bounds=[[0,10], [-1,1]])
        
        if res.fun < fmin and res.success:
            print('Successful fit')
            succ_res = res.copy()
            fmin = res.fun
            a, b = res.x
            hess_inv = res.hess_inv
    
    if succ_res is None:
        print('No successful fits')
    else:
        print(succ_res)
    
    return a, b, hess_inv

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4), tight_layout=True)

################################################################################
# Original data
#-------------------------------------------------------------------------------
print('Original')

ax.errorbar(rmag[isrmeas], 
            log10vmax[isrmeas], 
            yerr=dlog10vmax[isrmeas], 
            fmt='ko', 
            alpha=0.7,
            label='Abell 2151 data')

a, b, _ = fit_tfr(rmag[isrmeas], log10vmax[isrmeas], dlog10vmax[isrmeas])
print(a, b)

r = np.arange(12.5,18.6,0.1)
ax.plot(r, a + b*r, 'r--', alpha=0.7)
################################################################################


################################################################################
# Cleaned data
#-------------------------------------------------------------------------------
'''
print('\nCleaned')

rmag_clean = np.asarray(rmag_clean)
vmax_clean = np.asarray(vmax_clean)
dvmax_clean = np.asarray(dvmax_clean)

isrmeas_clean = rmag_clean > 0

log10vmax_clean = np.log10(vmax_clean)
dlog10vmax_clean = 0.434 * dvmax_clean / vmax_clean 

ax.errorbar(rmag_clean[isrmeas_clean], 
            log10vmax_clean[isrmeas_clean], 
            yerr=dlog10vmax_clean[isrmeas_clean], 
            fmt='ro', 
            alpha=0.7,
            label='Cleaned data, $\cos{(i)}$-corrected')

a, b, _ = fit_tfr(rmag_clean[isrmeas_clean], 
                  log10vmax_clean[isrmeas_clean], 
                  dlog10vmax_clean[isrmeas_clean])
print(a, b)
r = np.arange(12.5,18.6,0.1)
ax.plot(r, a + b*r, 'r--', alpha=0.8)
'''
################################################################################


ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       ylim=(-0.5,3),
       title=r'Rotational velocity of Abell 2151 in Daily Tiles')

ax.legend(loc='lower right', fontsize=10);

# fig.suptitle(r'Max velocity at $0.33\times R_{26}$', y=1.05)
# fig.subplots_adjust(top=0.8)
#fig.savefig('tf_coma.png', dpi=120)

# print(np.sum(isrmeas))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,5), tight_layout=True)

rmag_clean = np.asarray(rmag_clean)
vmax_clean = np.asarray(vmax_clean)
dvmax_clean = np.asarray(dvmax_clean)

isrmeas_clean = rmag_clean > 0

log10vmax_clean = np.log10(vmax_clean)
dlog10vmax_clean = 0.434 * dvmax_clean / vmax_clean

r = np.arange(12.5,18.6,0.1)
v = a + b*r

################################################################################
# Compute and plot the uncertainty range around the best fit
#-------------------------------------------------------------------------------
'''
hessian = ndt.Hessian(l1norm)
hess = hessian((a,b), 
               rmag_clean[isrmeas_clean], 
               log10vmax_clean[isrmeas_clean], 
               dlog10vmax_clean[isrmeas_clean])

N_samples = 1000

random_samples = np.random.multivariate_normal(mean=(a,b), 
                                               cov=np.linalg.inv(np.abs(hess)), #hess_inv.matmat(np.eye(2)), 
                                               size=N_samples)

y_samples = np.zeros([1000, len(r)])
for i in range(len(r)):
    y_samples[:,i] = random_samples[:,0] + random_samples[:,1]*r[i]

std_dev = np.std(y_samples, axis=0)

ax.fill_betweenx(r, v-std_dev, v+std_dev, facecolor='lightgray')
'''
################################################################################

ax.plot(v, r, 'k--', alpha=0.8)

ax.errorbar(log10vmax[isrmeas], 
            rmag[isrmeas], 
            xerr=dlog10vmax[isrmeas], 
            fmt='ro', 
            alpha=0.7)

ax.set(ylabel='$m_r$',
       ylim=(18.5, 12.5),
       xlabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       xlim=(1,3),
       title='Rotational velocity for Abell 2151 Cluster');

#plt.savefig('../Figures/PV_TFR_Coma_fitWOerrors_09262021.eps', format='eps', dpi=120);

In [ ]:
len(log10vmax[isrmeas])

#### Hyperfit

In [ ]:
# Create a 2x2xN matrix.
ndata = len(dlog10vmax[isrmeas])
cov = np.empty((2, 2, ndata))

# loop over arrays of uncertainties in logv and mag
# Assume diagonal covariance for each measurement.
for i, (dlogv, dm) in enumerate(zip(dlog10vmax[isrmeas], drmag[isrmeas])):
    cov[:,:,i] = np.array([[dlogv**2, 0.], [0., dm**2]])
                              
# cov[:,:,0]

In [ ]:
logv = log10vmax[isrmeas]
mr = rmag[isrmeas]
logv.shape, mr.shape

In [ ]:
data1 = [logv, mr]
cov1 = cov

In [ ]:
hf = LinFit([logv, mr], cov)

In [ ]:
# Run an MCMC
bounds = ((-10.0, 10.0), (-1000.0, 1000.0), (1.0e-5, 500.0))
mcmc_samples, mcmc_lnlike = hf.emcee(bounds, verbose=True)
print(np.mean(mcmc_samples, axis=1), np.std(mcmc_samples, axis=1))

# # Make the plot
# data.plot(linfit=hf)

In [ ]:
a, b, sig    = np.mean(mcmc_samples, axis=1)
da, db, dsig = np.std(mcmc_samples, axis=1)

for val, err in zip((a, b, sig), (da, db, dsig)):
    print('{:6.2f} +/- {:.2f}'.format(val, err))

In [ ]:
sigmas = hf.get_sigmas()
xvals = np.linspace(0., 3., 1000)
yvals = hf.coords[0] * xvals + hf.coords[1]

# Get the MCMC 1-sigma quantiles to plot with the fit.
y_chain = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[1]
y_chain_quantiles = np.quantile(y_chain, [0.1587, 0.8414], axis=1)

# Pack info into data
data = [log10vmax[isrmeas], rmag[isrmeas]]
x_err = dlog10vmax[isrmeas]
y_err = drmag[isrmeas]
corr_xy = np.zeros_like(x_err)

# Generate ellipses
ells = [
    Ellipse(
        xy=[data[0][i], data[1][i]],
        width=2.0 * y_err[i],
        height=2.0 * x_err[i],
        angle=np.rad2deg(np.arccos(corr_xy[i])),
    )
    for i in range(len(data[0]))
]

# Make the plot
fig = plt.figure(figsize=(6,7))
ax = fig.add_axes([0.15, 0.15, 1.03, 0.83])
for i, e in enumerate(ells):
    ax.add_artist(e)
    e.set_color(cmo.cm.haline(sigmas[i] / np.amax(sigmas)))
    e.set_edgecolor('None')
    e.set_alpha(1)
ax.fill_between(xvals, y_chain_quantiles[0], y_chain_quantiles[1], color="k", alpha=0.15)
ax.plot(xvals, yvals, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals - hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=1)
ax.plot(xvals, yvals + hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=1)
ax.set_xlabel(r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", fontsize=16)
ax.set_ylabel(r"$m_r$", fontsize=16)
ax.set_title(r"Abell 2151", fontsize=16)
ax.set_xlim(1.25, 2.75)
ax.set_ylim(18, 14.5)

# Add the colourbar
cb = fig.colorbar(
    cm.ScalarMappable(norm=colors.Normalize(vmin=0.0, vmax=np.amax(sigmas)), cmap = cmo.cm.haline),
    ax=ax,
    shrink=0.5,
    aspect=10,
    anchor=(-8, 0.95),
)
cb.set_label(label=r"$\sigma$", fontsize=14)

In [ ]:
import cmocean as cmo

In [ ]:
sigmas1 = hf.get_sigmas()

In [ ]:
vert1 = hf.vert_scat

In [ ]:
import corner

In [ ]:
fig = corner.corner(mcmc_samples.T, bins=30, smooth=1,
             range=[[-4, -12], [22, 40], [0.1, 1.3]],   # Range for a, b, sigma. Adjust as needed.
             labels=['$a$', '$b$', r'$\sigma$'],
             levels=(1-np.exp(-0.5), 1-np.exp(-2)),
             quantiles=[0.16, 0.5, 0.84],
             color='blue',
             hist_kwargs={'histtype':'stepfilled', 'alpha':0.3},
             plot_datapoints=False,
             fill_contours=True,
             show_titles=True,
             title_kwargs={"fontsize": 12})

### Virgo

In [ ]:
virgo_nest = 100002

virgo_row_t3 = table3['Nest'] == virgo_nest

R2t_virgo = table3['R2t'][virgo_row_t3][0]
sigma_virgo = table3['sigP'][virgo_row_t3][0]

In [ ]:
virgo_coords = SkyCoord(table3['SGLON'][virgo_row_t3]*u.degree, 
                       table3['SGLAT'][virgo_row_t3]*u.degree, 
                       frame='supergalactic')

v_group_coords = SkyCoord(table2['SGLON']*u.degree, 
                        table2['SGLAT']*u.degree, 
                        frame='supergalactic')

idx, d2d, d3d = virgo_coords.match_to_catalog_sky(v_group_coords)

V_virgo = table2['__HV_'][idx][0]

In [ ]:
# First, we need to convert R2t from Mpc to an angle, using the group's heliocentric velocity
R2t_virgo_angle = (R2t_virgo/(V_virgo/H0))*u.radian

In [ ]:
v_tf_coords = SkyCoord(SGA_main['TARGET_RA'], SGA_main['TARGET_DEC'], unit='deg')

vsep = virgo_coords.separation(v_tf_coords)

In [ ]:
fuji_in_virgo1 = (vsep < 1.5*R2t_virgo_angle) & (SGA_main['Z']*c > V_virgo - 3*sigma_virgo) & (SGA_main['Z']*c < V_virgo + 3*sigma_virgo)

fuji_in_virgo2 = (vsep >= 1.5*R2t_virgo_angle) & (vsep < 3*R2t_virgo_angle) & (SGA_main['Z']*c > V_virgo - 2*sigma_virgo) & (SGA_main['Z']*c < V_virgo + 2*sigma_virgo)

fuji_in_virgo = fuji_in_virgo1 | fuji_in_virgo2

################################################################################
# Keep all instances of each SGA_ID that are within the Coma cluster
#-------------------------------------------------------------------------------
fuji_ID_in_virgo = np.unique(SGA_main['SGA_ID_1'][fuji_in_virgo])

idx_fuji_in_virgo = np.in1d(SGA_main['SGA_ID_1'], fuji_ID_in_virgo)

inVirgo_fuji_table = SGA_main[idx_fuji_in_virgo]
################################################################################

inVirgo_fuji_table

In [ ]:
plt.hist(vsep[fuji_in_virgo].to_value('degree'))
plt.xlabel('MaNGA-NGC 4065 Angular Separation [deg]')
plt.ylabel('Count')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5), tight_layout=True)
ax = axes[0]
ax.plot(inVirgo_fuji_table['TARGET_DEC'], inVirgo_fuji_table['TARGET_RA'], '.')
ax = axes[1]
ax.hist(inVirgo_fuji_table['Z']);

#### Multiple Counts

In [ ]:
ids, counts = np.unique(inVirgo_fuji_table['SGA_ID_1'], return_counts=True)
ids[counts > 1]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,5), tight_layout=True)

ax.hist(counts[counts > 1], bins=np.linspace(2,30,34))

ax.set(xlabel='Observations per SGA_ID Virgo', 
       ylabel='count');

In [ ]:
# Keep only those with more than one observation
high_count_sga = np.in1d(inVirgo_fuji_table['SGA_ID_1'], ids[counts > 1])
tf_virgo_multiple = inVirgo_fuji_table[high_count_sga]
tf_virgo_multiple

#### Rotational Velocities

In [ ]:
sga_ids_vel_cuts = []

rmag = []
drmag = []
vmax = []
dvmax = []

for i, sga_id in enumerate(np.unique(inVirgo_fuji_table['SGA_ID_1'])):
    # if sga_id == 474614:
    #     print('skipped')
    #     continue
    galaxy_list = inVirgo_fuji_table[inVirgo_fuji_table['SGA_ID_1'] == sga_id]
    # print
    #print(i+1, sga_id)
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    r26 = 0.5 * float(sga_galaxy['D26'])*u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26
    
    zc, zc_err = sga_galaxy['Z'], sga_galaxy['ZERR']
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = c * dz
    dv_err = c * dz_err
    
    good_vel = dv < 5000
    # print(good_vel[0], sga_id)
    
    if np.sum(good_vel) > 0:
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26])
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2))

            rmag.append(float(sga_galaxy['R_MAG_SB26']))
            drmag.append(float(sga_galaxy['R_MAG_SB26_ERR']))
            vmax.append(v033)
            dvmax.append(dv033)
            sga_ids_vel_cuts.append(sga_id)
    # break
#print('mag:', rmag)
#print('vel:', vmax)
#print('dv:', dvmax)

In [ ]:
rmag = np.asarray(rmag)
drmag = np.asarray(drmag)
vmax = np.asarray(vmax)
dvmax = np.asarray(dvmax)

isrmeas = rmag > 0

fig, axes = plt.subplots(1, 2, figsize=(10,4), tight_layout=True)

ax = axes[0]
ax.errorbar(rmag[isrmeas], vmax[isrmeas], xerr=drmag[isrmeas], yerr=dvmax[isrmeas], fmt='ro')
ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylim=(-50,300),
       ylabel='$v_\mathrm{max}$ [km s$^{-1}$]')

ax = axes[1]

log10vmax = np.log10(vmax)
dlog10vmax = 0.434 * dvmax / vmax 

ax.errorbar(rmag[isrmeas], log10vmax[isrmeas], yerr=dlog10vmax[isrmeas], fmt='ro')
ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       ylim=(-0.5,3))

fig.suptitle(r'Rotational velocity at $0.4R_{26}$ for Virgo', y=1.05)
fig.subplots_adjust(top=0.8)

print(np.sum(isrmeas))

In [ ]:
sga_ids_clean = []

rmag_clean = []
drmag_clean = []
vmax_clean = []
dvmax_clean = []
z = []

inc_min = 45*u.degree
cosi_max = np.cos(inc_min.to('radian'))

inVirgo_fuji_table['cosi'] = np.sqrt((inVirgo_fuji_table['BA']**2 - q0**2)/(1 - q0**2))
inVirgo_fuji_table['cosi'][np.isnan(inVirgo_fuji_table['cosi'])] = 0 # Objects with b/a < 0.2

#i = 0

# for sga_id in np.unique(inComa_sga_table['SGA_ID']):
for sga_id in sga_ids_vel_cuts:
    galaxy_list = inVirgo_fuji_table[inVirgo_fuji_table['SGA_ID_1'] == sga_id]
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    
    morphtype = str(sga_galaxy['MORPHTYPE'])
    
    cosi = float(sga_galaxy['cosi'])
    
    r26 = 0.5 * float(sga_galaxy['D26']) * u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26

    # Cut any suspected ellipticals
    if morphtype.startswith('E') or morphtype.startswith('S0') or morphtype.startswith('I'):
        continue
               
    # Inclination cut
    if cosi > cosi_max:
        continue
        
    #i += 1
    #print(i, sga_id, cosi)
    
    zc, zc_err = float(sga_galaxy['Z']), float(sga_galaxy['ZERR'])
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = 3e5 * dz
    dv_err = 3e5 * dz_err
    
    good_vel = dv < 5000
    
    if np.sum(good_vel) > 0:
    
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26]) / np.sqrt(1 - cosi**2)
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2)) / np.sqrt(1 - cosi**2)
            z.append(zc)
            rmag_clean.append(float(sga_galaxy['R_MAG_SB26']))
            drmag_clean.append(float(sga_galaxy['R_MAG_SB26_ERR']))
            vmax_clean.append(v033)
            dvmax_clean.append(dv033)
            sga_ids_clean.append(sga_id)
    
#inComa_sga_table[['SGA_ID', 'BA', 'cosi']].show_in_notebook()
print(len(rmag_clean))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4), tight_layout=True)

################################################################################
# Original data
#-------------------------------------------------------------------------------
print('Original')

ax.errorbar(rmag[isrmeas], 
            log10vmax[isrmeas], 
            yerr=dlog10vmax[isrmeas], 
            fmt='ko', 
            alpha=0.7,
            label='Abell 2151 data')

a, b, _ = fit_tfr(rmag[isrmeas], log10vmax[isrmeas], dlog10vmax[isrmeas])
print(a, b)

r = np.arange(12.5,18.6,0.1)
ax.plot(r, a + b*r, 'r--', alpha=0.7)
################################################################################


################################################################################
# Cleaned data
#-------------------------------------------------------------------------------
'''
print('\nCleaned')

rmag_clean = np.asarray(rmag_clean)
vmax_clean = np.asarray(vmax_clean)
dvmax_clean = np.asarray(dvmax_clean)

isrmeas_clean = rmag_clean > 0

log10vmax_clean = np.log10(vmax_clean)
dlog10vmax_clean = 0.434 * dvmax_clean / vmax_clean 

ax.errorbar(rmag_clean[isrmeas_clean], 
            log10vmax_clean[isrmeas_clean], 
            yerr=dlog10vmax_clean[isrmeas_clean], 
            fmt='ro', 
            alpha=0.7,
            label='Cleaned data, $\cos{(i)}$-corrected')

a, b, _ = fit_tfr(rmag_clean[isrmeas_clean], 
                  log10vmax_clean[isrmeas_clean], 
                  dlog10vmax_clean[isrmeas_clean])
print(a, b)
r = np.arange(12.5,18.6,0.1)
ax.plot(r, a + b*r, 'r--', alpha=0.8)
'''
################################################################################


ax.set(xlabel='$m_r$(26)',
       xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       ylim=(-0.5,3),
       title=r'Rotational velocity of Virgo in Daily Tiles')

ax.legend(loc='lower right', fontsize=10);

# fig.suptitle(r'Max velocity at $0.33\times R_{26}$', y=1.05)
# fig.subplots_adjust(top=0.8)
#fig.savefig('tf_coma.png', dpi=120)

# print(np.sum(isrmeas))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,5), tight_layout=True)

rmag_clean = np.asarray(rmag_clean)
vmax_clean = np.asarray(vmax_clean)
dvmax_clean = np.asarray(dvmax_clean)

isrmeas_clean = rmag_clean > 0

log10vmax_clean = np.log10(vmax_clean)
dlog10vmax_clean = 0.434 * dvmax_clean / vmax_clean

r = np.arange(12.5,18.6,0.1)
v = a + b*r

################################################################################
# Compute and plot the uncertainty range around the best fit
#-------------------------------------------------------------------------------
'''
hessian = ndt.Hessian(l1norm)
hess = hessian((a,b), 
               rmag_clean[isrmeas_clean], 
               log10vmax_clean[isrmeas_clean], 
               dlog10vmax_clean[isrmeas_clean])

N_samples = 1000

random_samples = np.random.multivariate_normal(mean=(a,b), 
                                               cov=np.linalg.inv(np.abs(hess)), #hess_inv.matmat(np.eye(2)), 
                                               size=N_samples)

y_samples = np.zeros([1000, len(r)])
for i in range(len(r)):
    y_samples[:,i] = random_samples[:,0] + random_samples[:,1]*r[i]

std_dev = np.std(y_samples, axis=0)

ax.fill_betweenx(r, v-std_dev, v+std_dev, facecolor='lightgray')
'''
################################################################################

ax.plot(v, r, 'k--', alpha=0.8)

ax.errorbar(log10vmax[isrmeas], 
            rmag[isrmeas], 
            xerr=dlog10vmax[isrmeas], 
            fmt='ro', 
            alpha=0.7)

ax.set(ylabel='$m_r$',
       ylim=(18.5, 12.5),
       xlabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       xlim=(1,3),
       title='Rotational velocity for Abell 2151 Cluster');

#plt.savefig('../Figures/PV_TFR_Coma_fitWOerrors_09262021.eps', format='eps', dpi=120);

In [ ]:
len(log10vmax[isrmeas])

#### Hyperfit

In [ ]:
# Create a 2x2xN matrix.
ndata = len(dlog10vmax[isrmeas])
cov = np.empty((2, 2, ndata))

# loop over arrays of uncertainties in logv and mag
# Assume diagonal covariance for each measurement.
for i, (dlogv, dm) in enumerate(zip(dlog10vmax[isrmeas], drmag[isrmeas])):
    cov[:,:,i] = np.array([[dlogv**2, 0.], [0., dm**2]])
                              
# cov[:,:,0]

In [ ]:
logv = log10vmax[isrmeas]
mr = rmag[isrmeas]
logv.shape, mr.shape

In [ ]:
data2 = [logv, mr]
cov2 = cov

In [ ]:
hf = LinFit([logv, mr], cov)

In [ ]:
hf

In [ ]:
# Run an MCMC
bounds = ((-6.0, 6.0), (-1000.0, 1000.0), (1.0e-5, 500.0))
mcmc_samples, mcmc_lnlike = hf.emcee(bounds, verbose=True)
print(np.mean(mcmc_samples, axis=1), np.std(mcmc_samples, axis=1))

# # Make the plot
# data.plot(linfit=hf)

In [ ]:
a, b, sig    = np.mean(mcmc_samples, axis=1)
da, db, dsig = np.std(mcmc_samples, axis=1)

for val, err in zip((a, b, sig), (da, db, dsig)):
    print('{:6.2f} +/- {:.2f}'.format(val, err))

In [ ]:
sigmas = hf.get_sigmas()
xvals = np.linspace(0., 3., 1000)
yvals = hf.coords[0] * xvals + hf.coords[1]

# Get the MCMC 1-sigma quantiles to plot with the fit.
y_chain = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[1]
y_chain_quantiles = np.quantile(y_chain, [0.1587, 0.8414], axis=1)

# Pack info into data
data = [log10vmax[isrmeas], rmag[isrmeas]]
x_err = dlog10vmax[isrmeas]
y_err = drmag[isrmeas]
corr_xy = np.zeros_like(x_err)

# Generate ellipses
ells = [
    Ellipse(
        xy=[data[0][i], data[1][i]],
        width=2.0 * y_err[i],
        height=2.0 * x_err[i],
        angle=np.rad2deg(np.arccos(corr_xy[i])),
    )
    for i in range(len(data[0]))
]

# Make the plot
fig = plt.figure(figsize=(6,7))
ax = fig.add_axes([0.15, 0.15, 1.03, 0.83])
for i, e in enumerate(ells):
    ax.add_artist(e)
    e.set_color(cmo.cm.haline(sigmas[i] / np.amax(sigmas)))
    e.set_edgecolor('None')
    e.set_alpha(0.9)
ax.fill_between(xvals, y_chain_quantiles[0], y_chain_quantiles[1], color="k", alpha=0.2)
ax.plot(xvals, yvals, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals - hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals + hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.set_xlabel(r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", fontsize=16)
ax.set_ylabel(r"$m_r$", fontsize=16)
ax.set_title(r"Virgo", fontsize=16)
ax.set_xlim(0, 3)
ax.set_ylim(18, 12)

# Add the colourbar
cb = fig.colorbar(
    cm.ScalarMappable(norm=colors.Normalize(vmin=0.0, vmax=np.amax(sigmas)), cmap = cmo.cm.haline),
    ax=ax,
    shrink=0.5,
    aspect=10,
    anchor=(-8, 0.95),
)
cb.set_label(label=r"$\sigma$", fontsize=14)

In [ ]:
sigma2 = hf.get_sigmas()

In [ ]:
vert2 = hf.vert_scat

In [ ]:
import cmocean

In [ ]:
fig = corner.corner(mcmc_samples.T, bins=30, smooth=1,
             range=[[-1, -9], [16, 31], [0.2, 2.5]],   # Range for a, b, sigma. Adjust as needed.
             labels=['$a$', '$b$', r'$\sigma$'],
             levels=(1-np.exp(-0.5), 1-np.exp(-2)),
             quantiles=[0.16, 0.5, 0.84],
             color='blue',
             hist_kwargs={'histtype':'stepfilled', 'alpha':0.3},
             plot_datapoints=False,
             fill_contours=True,
             show_titles=True,
             title = {'Coma'},
             title_kwargs={"fontsize": 12})

### Abell 2151 and Virgo

#### Chi Squared

In [ ]:
def chi2(params, data1, data2, cov1, cov2):
    """Chi-square function for joint slope fit to two data sets.
    
    Parameters
    ----------
    data1 : ndarray
        2xN array of [x1, y1] for data set 1.
    data2 : ndarray
        2xM array of [x2, y2] for data set 2.
    cov1 : ndarray
        2x2xN covariances for data set 1.
    cov2 : ndarray
        2x2xM covariances for data set 2.
        
    Returns
    -------
    chi2 : float
        Sum of chi-square fits to data sets 1 and 2.
    """
    a, b1, b2 = params
    
    x1, y1 = data1[0], data1[1]
    varx1, vary1 = cov1[0,0], cov1[1,1]
    chi2_1 = np.sum((y1 - a*x1 - b1)**2 / (vary1 + a**2*varx1))
    
    x2, y2 = data2[0], data2[1]
    varx2, vary2 = cov2[0,0], cov2[1,1]
    chi2_2 = np.sum((y2 - a*x2 - b2)**2 / (vary2 + a**2*varx2))
    
    return chi2_1 + chi2_2



In [ ]:
p0 = [-6, 31, 22]
res = minimize(chi2, p0, args=(data1, data2, cov1, cov2), method='BFGS')
res

In [ ]:
a_, b1_, b2_ = res.x
da_, db1_, db2_ = [np.sqrt(res.hess_inv[i,i]) for i in range(3)]

fig, ax = plt.subplots(1,1, figsize=(8,5))
ax.invert_xaxis()
eb1 = ax.errorbar(data1[0], data1[1], xerr=np.sqrt(cov1[0,0]), yerr=np.sqrt(cov1[1,1]), fmt='.')
ax.plot(data1[0], a_*data1[0] + b1_, color=eb1[0].get_color(),
        label=r'$\hat{{a}}={:.2f}\pm{:.2f}$, $\hat{{b}}_1={:.2f}\pm{:.2f}$'.format(a_, da_, b1_, db1_))

eb2 = ax.errorbar(data2[0], data2[1], xerr=np.sqrt(cov2[0,0]), yerr=np.sqrt(cov2[1,1]), fmt='.')
ax.plot(data2[0], a_*data2[0] + b2_, color=eb2[0].get_color(),
        label=r'$\hat{{a}}={:.2f}\pm{:.2f}$, $\hat{{b}}_1={:.2f}\pm{:.2f}$'.format(a_, da_, b2_, db2_))

ax.set(xlabel='$x$', ylabel='$y$',
       title='$a={:g}$, $b_1={:g}$, $b_2={:g}$'.format(a_, b1_, b2_))
ax.legend(fontsize=12);

#### LinFit

In [ ]:
from scipy.optimize import minimize, differential_evolution
import emcee
from hyperfit.linfit import LinFit

In [ ]:
def nlogl(params, datasets, covs):
    """Chi-square function for joint slope fit to two or more data sets.
    
    Parameters
    ----------
    datasets : list or ndarray
        m x 2xN array of [x1, y1] for each data set.
    cov : ndarray
        m x 2x2xN covariances for each data set.
        
    Returns
    -------
    chi2 : float
        Sum of chi-square fits to data sets 1 and 2.
    """
    nsets = len(datasets)
    a = params[0]
    b = params[1:nsets+1]
    sigma = params[nsets+1:]
    
    nloglike = 0.
    for i in range(nsets): 
        data = datasets[i]
        cov = covs[i]
        x, dx2 = data[0], cov[0,0]
        y, dy2 = data[1], cov[1,1]
        dxy = cov[0,1]
        sy2 = sigma[i]**2 + a**2*dx2 + dy2 - 2*dxy*a
        nloglike += -0.5*np.sum(np.log((a**2 + 1)/sy2) - (a*x - y + b[i])**2/sy2)
    
    return nloglike

# Minimization.
print('Differential evolution:')
bounds = [[-10., 10.], [-6., 6.], [-5., 5.], [0, 1], [0, 2]]
res = differential_evolution(nlogl, bounds, args=([data1, data2], [cov1, cov2]))
print(res)

# Initial guesses
slope = -6
intercepts = [31, 22]
sigmas = [0.2, 0.3]
p0 = [slope] + intercepts + sigmas

print('\n\nBFGS minimization:')
res = minimize(nlogl, p0, args=([data1, data2], [cov1, cov2]), method='BFGS')
print(res) 

In [ ]:
a_, b1_, b2_, sig1_, sig2_ = res.x
da_, db1_, db2_ = [np.sqrt(res.hess_inv[i,i]) for i in range(3)]


fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.invert_xaxis()
eb1 = ax.errorbar(data1[0], data1[1], xerr=np.sqrt(cov1[0,0]), yerr=np.sqrt(cov1[1,1]), fmt='.')
x_ = np.linspace(np.min(data1[0]), np.max(data1[0]))
ax.plot(x_, a_*x_ + b1_, color=eb1[0].get_color(),
        label=r'Abell 2151: $\hat{{a}}={:.2f}\pm{:.2f}$, $\hat{{b}}_1={:.2f}\pm{:.2f}$'.format(a_, da_, b1_, db1_))
ax.plot(x_, a_*x_ + b1_ + sig1_, ls=':', color=eb1[0].get_color())
ax.plot(x_, a_*x_ + b1_ - sig1_, ls=':', color=eb1[0].get_color())

eb2 = ax.errorbar(data2[0], data2[1], xerr=np.sqrt(cov2[0,0]), yerr=np.sqrt(cov2[1,1]), fmt='.')

x_ = np.linspace(np.min(data2[0]), np.max(data2[0]))
ax.plot(data2[0], a_*data2[0] + b2_, color=eb2[0].get_color(),
        label=r'Virgo: $\hat{{a}}={:.2f}\pm{:.2f}$, $\hat{{b}}_1={:.2f}\pm{:.2f}$'.format(a_, da_, b2_, db2_))
ax.plot(x_, a_*x_ + b2_ + sig2_, ls=':', color=eb2[0].get_color())
ax.plot(x_, a_*x_ + b2_ - sig2_, ls=':', color=eb2[0].get_color())
ax.set_ylabel(r"$m_r$", fontsize = 15.0)
ax.set_xlabel(r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", fontsize= 15.0)
ax.set_title('Virgo and Abell 2151', fontsize = 15.0)

#ax.set(xlabel=r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", ylabel=r"$m_r$",
       #title='Virgo and Abell 2151', fontsize = 20.0)
ax.legend(fontsize=12);

In [ ]:
class MultiLinFit:
    
    def __init__(self, datasets, covs, weights=None, vertaxis=-1):
        
        self.nsets = len(datasets)
        self.ndims = np.shape(datasets[0])[0]
        self.ndata = [np.shape(data)[1] for data in datasets]
        self.datasets = datasets
        self.covs = covs
        self.data = None
        self.cov = None
        
        self.npars = 1 + self.nsets # slope + intercepts + sigmas
        self.params = np.zeros(self.npars)
        self.params_scatter = np.zeros(self.nsets)
        
        self.weights = [np.ones(n) for n in self.ndata] if weights is None else weights
        self.vertaxis = vertaxis
        
        self.param_bounds = None      # parameter fit bounds for all data sets
        
    def _lnpost(self, params):
        lnpost = 0.

        for i in range(self.nsets):
            # Loop over individual data sets. 
            self.data = self.datasets[i]
            self.cov  = self.covs[i]
            
            # Set up parameter and bounds arrays for each data set.
            pars_i = np.array([params[0]] + [params[1+i]] + [params[self.nsets+1+i]])
            bounds_i = [self.param_bounds[0]] + \
                       [self.param_bounds[1+i]] + \
                       [self.param_bounds[self.nsets+1+i]]

            # Set up weights for each data set.
            weights = self.weights[i]
            
            # Sum over all data sets.
            lnprior = self._lnprior(pars_i, bounds_i)
            lnlike = self._lnlike(pars_i)                
            lnpost += np.sum(weights * lnlike) + lnprior
        
        return lnpost
                    
    def _lnprior(self, params, bounds):
        lnprior = 0.
        for i, (param, bound) in enumerate(zip(params.T, bounds)):
            lnprior += np.where(np.logical_or(param < bound[0], param > bound[1]), -np.inf, 0.0)

        return lnprior
    
    def _lnlike(self, params):
        a, b, sigma = params

        x, dx2 = self.data[0], self.cov[0,0]
        y, dy2 = self.data[1], self.cov[1,1]
        dxy = self.cov[0,1]
        sy2 = sigma**2 + a**2*dx2 + dy2 - 2*dxy*a
        lnlike = 0.5*np.sum(np.log((a**2 + 1)/sy2) - (a*x - y + b)**2/sy2)

        return lnlike
    
    def optimize(self, bounds, tol=1e-6, verbose=False):
        self.param_bounds = bounds
        res = differential_evolution(lambda *args: -self._lnpost(*args), self.param_bounds, tol=tol)

        if verbose:
            print(res)
            
        self.params = res.x[:-self.nsets]
        self.params_scatter = np.fabs(res.x[-self.nsets:])
        return self.params, self.params_scatter, res.fun
    
    def emcee(self, bounds, max_iter=100000, batchsize=1000, ntau=50.0, tautol=0.05, verbose=False):

        # Set up emcee. Start the walkers in a small 1 percent ball around the best fit.
        # The best fit will set self.params and self.params_scatter.
        self.optimize(bounds, verbose=verbose)
        ndim = len(self.params) + len(self.params_scatter)
        nwalker = 4 * ndim
        seeds = np.asarray([
            [(0.01 * np.random.rand() + 0.995) * j for j in np.concatenate([self.params, self.params_scatter])]
            for _ in range(nwalker)
        ])

        sampler = emcee.EnsembleSampler(nwalker, ndim, self._lnpost)

        old_tau = np.inf
        niter = 0
        converged = 0
        while ~converged:
            sampler.run_mcmc(seeds, nsteps=batchsize, progress=verbose)
            tau = sampler.get_autocorr_time(discard=int(0.5 * niter), tol=0)
            converged = np.all(ntau * tau < niter)
            converged &= np.all(np.abs(old_tau - tau) / tau < tautol)
            old_tau = tau
            begin = None
            niter += 1000
            if verbose:
                print("Niterations/Max Iterations: ", niter, "/", max_iter)
                print("Integrated ACT/Min Convergence Iterations: ", tau, "/", np.amax(ntau * tau))
            if niter >= max_iter:
                break

        # Remove burn-in and and save the samples
        tau = sampler.get_autocorr_time(discard=int(0.5 * niter), tol=0)
        burnin = int(2 * np.max(tau))
        samples = sampler.get_chain(discard=burnin, flat=True).T
        mcmc_samples = samples
        mcmc_lnlike = sampler.get_log_prob(discard=burnin, flat=True)

        return mcmc_samples, mcmc_lnlike

In [ ]:
mlf = MultiLinFit([data1, data2], [cov1, cov2])

In [ ]:
bounds = [[-10., 10.], [-10., 10.], [-10., 10.], [0., 2.], [0., 2.]]
mlf.optimize(bounds, verbose=True)

In [ ]:
# Run an MCMC
bounds = [[-10., 10.], [-10., 40.], [-10., 30.], [0., 2.], [0., 2.]]
mcmc_samples, mcmc_lnlike = mlf.emcee(bounds, max_iter=10000, verbose=True)
print(np.mean(mcmc_samples, axis=1), np.std(mcmc_samples, axis=1))

In [ ]:
a, b1, b2, sig1, sig2    = np.mean(mcmc_samples, axis=1)
da, db1, db2, dsig1, dsig2 = np.std(mcmc_samples, axis=1)

for val, err in zip((a, b1, b2, sig1, sig2), (da, db1, db2, dsig1, dsig2)):
    print('{:6.2f} +/- {:.2f}'.format(val, err))

In [ ]:
xvals = np.linspace(0., 3., 1000)
yvals1 = a * xvals + b1
yvals2 = a * xvals + b2

# Get the MCMC 1-sigma quantiles to plot with the fit.
y_chain1 = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[1]
y_chain_quantiles1 = np.quantile(y_chain1, [0.1587, 0.8414], axis=1)

y_chain2 = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[2]
y_chain_quantiles2 = np.quantile(y_chain2, [0.1587, 0.8414], axis=1)

# Pack info into data
x_err1 = np.sqrt(cov1[0][0])
y_err1 = np.sqrt(cov2[1][1])
corr_xy1 = np.zeros_like(x_err1)

x_err2 = np.sqrt(cov2[0][0])
y_err2 = np.sqrt(cov2[1][1])
corr_xy2 = np.zeros_like(x_err2)


# Generate ellipses
ells1 = [
    Ellipse(
        xy=[data1[0][i], data1[1][i]],
        width=2.0 * y_err1[i],
        height=2.0 * x_err1[i],
        angle=np.rad2deg(np.arccos(corr_xy1[i])),
    )
    for i in range(len(data1[0]))
]

ells2 = [
    Ellipse(
        xy=[data2[0][i], data2[1][i]],
        width=2.0 * y_err2[i],
        height=2.0 * x_err2[i],
        angle=np.rad2deg(np.arccos(corr_xy2[i])),
    )
    for i in range(len(data2[0]))
]


print(len(ells1))
print(len(ells2))


# Make the plot
fig = plt.figure(figsize=(12,15))
ax = fig.add_axes([0.15, 0.15, 1.03, 0.83])
for i, e in enumerate(ells1):
    ax.add_artist(e)
    e.set_color('blue')
    e.set_edgecolor('blue')
    e.set_alpha(0.9)

e.set_label('Abell 2151')
for i, e in enumerate(ells2):
    ax.add_artist(e)
    e.set_color('red')
    e.set_edgecolor('red')
    e.set_alpha(0.9)

e.set_label('Virgo')
ax.fill_between(xvals, y_chain_quantiles1[0], y_chain_quantiles1[1], color="k", alpha=0.2)
ax.fill_between(xvals, y_chain_quantiles2[0], y_chain_quantiles2[1], color="k", alpha=0.2)
ax.plot(xvals, -6.95*xvals + b1, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, -6.95*xvals + b1 - vert1, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, -6.95*xvals + b1 + vert1, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, -6.95*xvals + b2, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, -6.95*xvals + b2 - vert2, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, -6.95*xvals + b2 + vert2, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.set_xlabel(r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", fontsize=32)
ax.set_ylabel(r"$m_r$", fontsize=32)
ax.set_title(r"Virgo and Abell 2151", fontsize=32)
ax.set_xlim(0.5, 2.75)
ax.set_ylim(18, 12.5)

ax.legend(fontsize=26)

# Add the colourbar
#cb = fig.colorbar(
 #   cm.ScalarMappable(norm=colors.Normalize(vmin=0.0, vmax=np.amax(sigmas)), cmap = cmo.cm.matter),
  #  ax=ax,
   # shrink=0.4,
    #aspect=10,
    #anchor=(-7.2, 0.95),
#)
#cb.set_label(label=r"$\sigma$", fontsize=14)

plt.savefig('virgo&a2151.png', bbox_inches='tight')

In [ ]:
from scipy.stats import linregress

In [ ]:
linregress(xvals, -6.95*xvals + b1)

In [ ]:
sigmas = hf.get_sigmas()
xvals = np.linspace(0., 3., 1000)
yvals = hf.coords[0] * xvals + hf.coords[1]

# Get the MCMC 1-sigma quantiles to plot with the fit.
y_chain = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[1]
y_chain_quantiles = np.quantile(y_chain, [0.1587, 0.8414], axis=1)

# Pack info into data
data = [log10vmax[isrmeas], rmag[isrmeas]]
x_err = dlog10vmax[isrmeas]
y_err = drmag[isrmeas]
corr_xy = np.zeros_like(x_err)

# Generate ellipses
ells = [
    Ellipse(
        xy=[data[0][i], data[1][i]],
        width=2.0 * y_err[i],
        height=2.0 * x_err[i],
        angle=np.rad2deg(np.arccos(corr_xy[i])),
    )
    for i in range(len(data[0]))
]

# Make the plot
fig = plt.figure(figsize=(6,7))
ax = fig.add_axes([0.15, 0.15, 1.03, 0.83])
for i, e in enumerate(ells):
    ax.add_artist(e)
    e.set_color(cmo.cm.haline(sigmas[i] / np.amax(sigmas)))
    e.set_edgecolor('None')
    e.set_alpha(0.9)
ax.fill_between(xvals, y_chain_quantiles[0], y_chain_quantiles[1], color="k", alpha=0.2)
ax.plot(xvals, yvals, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals - hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals + hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.set_xlabel(r"$\log_{10}{(v_\mathrm{0.4R_{26}} / \mathrm{km~s^{-1}})}$", fontsize=16)
ax.set_ylabel(r"$m_r$", fontsize=16)
ax.set_title(r"Virgo", fontsize=16)
ax.set_xlim(0, 3)
ax.set_ylim(18, 12)

# Add the colourbar
cb = fig.colorbar(
    cm.ScalarMappable(norm=colors.Normalize(vmin=0.0, vmax=np.amax(sigmas)), cmap = cmo.cm.haline),
    ax=ax,
    shrink=0.5,
    aspect=10,
    anchor=(-8, 0.95),
)
cb.set_label(label=r"$\sigma$", fontsize=14)

In [ ]:
fig = corner.corner(mcmc_samples.T, bins=25, smooth=1,
#              range=[[1.9, 2.4], [0.75, 1.1], [0.1, 0.3]],   # Range for a, b, sigma. Adjust as needed.
             labels=['$a$', '$b_1$', '$b_2$', r'$\sigma_1$', r'$\sigma_2$'],
             levels=(1-np.exp(-0.5), 1-np.exp(-2)),
             quantiles=[0.16, 0.5, 0.84],
             color='blue',
             hist_kwargs={'histtype':'stepfilled', 'alpha':0.3},
             plot_datapoints=False,
             fill_contours=True,
             show_titles=True,
             title_kwargs={"fontsize": 12})
plt.savefig('cornerplot.png')

### Calibration 

In [ ]:
SGA_IDs = tdaily['SGA_ID']

In [ ]:
SGA = Table.read('/global/homes/h/hnofi/DESI_SGA/TF/SGA_distances.fits', format = 'fits')

In [ ]:
t = join(SGA_main, SGA, keys_left='SGA_ID_1', keys_right='SGA_ID')
t

In [ ]:
for i in range(0, len(t)):
    if (t['SN_Catalog'][i] == '-1') & (t['Stellar_Catalog'][i] == '-1'):
        t.remove_row(i)

In [ ]:
# All targets
_ids_all, _counts_all = np.unique(t['SGA_ID'], return_counts=True)
_ids_all[_counts_all > 1]

#### Remove Galaxy: 501697

In [ ]:
for i in range(0, len(t)):
    if (t['SGA_ID_1'][i] == 501697):
        t.remove_row(i)

In [ ]:
sga_ids_vel_cuts = []
rmag = []
rmag_err = []
abs_mag = []
abs_mag_err = []
vmax = []
dvmax = []

for i, sga_id in enumerate(t['SGA_ID_1']):
    # if sga_id == 474614:
    #     print('skipped')
    #     continue
    galaxy_list = t[t['SGA_ID_1'] == sga_id]
    print
    #print(i+1, sga_id)
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    r26 = 0.5 * float(sga_galaxy['D26_1'])*u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26
    
    zc, zc_err = sga_galaxy['Z'], sga_galaxy['ZERR']
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = c * dz
    dv_err = c * dz_err
    
    good_vel = dv < 5000
    
    if np.sum(good_vel) > 0:
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26])
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2))

            rmag.append(float(sga_galaxy['R_MAG_SB26_1']))
            rmag_err.append(float(sga_galaxy['R_MAG_SB26_ERR_1']))
            vmax.append(v033)
            dvmax.append(dv033)
            # Find absolute magnitude - 5log(h), using h=0.742 (Union2 (2010))
            abs_mag.append(float(sga_galaxy['R_MAG_SB26_1'] - sga_galaxy['DM1_SN']) - 5*np.log10(0.742))
            abs_mag_err.append(np.sqrt((float(sga_galaxy['R_MAG_SB26_ERR_1']))**2 + (float(sga_galaxy['e_DM1_SN']))**2))
            sga_ids_vel_cuts.append(sga_id)
            
#print(sga_ids_vel_cuts)
#print('mag:', rmag)
#print('vel:', vmax)
#print('dv:', dvmax)

In [ ]:
rmag = np.asarray(rmag)
rmag_err = np.asarray(rmag_err)
vmax = np.asarray(vmax)
dvmax = np.asarray(dvmax)
abs_mag = np.asarray(abs_mag)
abs_mag_err = np.asarray(abs_mag_err)

isrmeas = rmag > 0

fig, axes = plt.subplots(1, 2, figsize=(10,4), tight_layout=True)

ax = axes[0]
ax.errorbar(rmag[isrmeas], vmax[isrmeas], xerr = rmag_err[isrmeas], yerr=dvmax[isrmeas], fmt='ro')
ax.set(xlabel='$M_r$(26) - 5log$_{10}$(h)',
       #xlim=(18.5, 12.5),
       #ylim=(-50,300),
       ylabel='$v_\mathrm{max}$ [km s$^{-1}$]')

ax = axes[1]

log10vmax = np.log10(vmax)
dlog10vmax = 0.434 * dvmax / vmax 

ax.errorbar(rmag[isrmeas], log10vmax[isrmeas], xerr = rmag_err[isrmeas], yerr=dlog10vmax[isrmeas], fmt='ro')
ax.set(xlabel='$M_r$(26) - 5log$_{10}$(h)',
       #xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$')
       #ylim=(-0.5,3))

fig.suptitle(r'Rotational velocity at $0.4R_{26}$', y=1.05)
fig.subplots_adjust(top=0.8)

print(np.sum(isrmeas))

In [ ]:
rmag = np.asarray(rmag)
vmax = np.asarray(vmax)
dvmax = np.asarray(dvmax)
abs_mag = np.asarray(abs_mag)
abs_mag_err = np.asarray(abs_mag_err)
# abs_mag_err = np.sqrt()

fig, axes = plt.subplots(1, 2, figsize=(10,4), tight_layout=True)

ax = axes[0]
ax.errorbar(abs_mag, vmax, xerr = abs_mag_err,yerr=dvmax, fmt='ro')
ax.set(xlabel='$M_r$(26)',
       # xlim=(18.5, 12.5),
       ylim=(-50,300),
       ylabel='$v_\mathrm{max}$ [km s$^{-1}$]')

ax = axes[1]

log10vmax = np.log10(vmax)
dlog10vmax = 0.434 * dvmax / vmax 

ax.errorbar(abs_mag[isrmeas], log10vmax[isrmeas], xerr = abs_mag_err[isrmeas],yerr=dlog10vmax[isrmeas], fmt='ro')
ax.set(xlabel='$M_r$(26)',
       # xlim=(18.5, 12.5),
       ylabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
       ylim=(-0.5,3))

fig.suptitle(r'Rotational velocity at $0.4R_{26}$', y=1.05)
fig.subplots_adjust(top=0.8)

print(np.sum(isrmeas))

In [ ]:
sga_table = t

In [ ]:
rmag_clean = []
rmag_err_clean = []
vmax_clean = []
dvmax_clean = []
abs_mag_clean = []
abs_mag_err_clean = []
z= []

inc_min = 45*u.degree
cosi_max = np.cos(inc_min.to('radian'))

sga_table['cosi'] = np.sqrt((sga_table['BA_1']**2 - q0**2)/(1 - q0**2))
sga_table['cosi'][np.isnan(sga_table['cosi'])] = 0 # Objects with b/a < 0.2

#i = 0

for sga_id in np.unique(sga_table['SGA_ID_1']):
    galaxy_list = sga_table[sga_table['SGA_ID_1'] == sga_id]
    
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    
    if np.sum(is_sga_galaxy) > 1:
        sga_galaxy = sga_galaxy[0]
    
    targetid = int(sga_galaxy['TARGETID'])
    
    center = SkyCoord(sga_galaxy['TARGET_RA'], sga_galaxy['TARGET_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['TARGET_RA'], tf_list['TARGET_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    
    morphtype = str(sga_galaxy['MORPHTYPE_1'])
    
    cosi = float(sga_galaxy['cosi'])
    
    r26 = 0.5 * float(sga_galaxy['D26_1']) * u.arcmin
    sep_r26 = sep2d.to('arcmin') / r26

    # Cut any suspected ellipticals
    if morphtype.startswith('E') or morphtype.startswith('S0') or morphtype.startswith('I'):
        print('{} cut (morphology)'.format(sga_id))
        continue
               
    # Inclination cut
    if cosi > cosi_max:
        print('{} cut (inclination)'.format(sga_id))
        continue
        
    #i += 1
    #print(i, sga_id, cosi)
    
    zc, zc_err = float(sga_galaxy['Z']), float(sga_galaxy['ZERR'])
    zt, zt_err = tf_list['Z'], tf_list['ZERR']
    
    dz = np.abs(zt - zc)
    dz_err = np.sqrt(zc_err**2 + zt_err**2)
    
    dv = 3e5 * dz
    dv_err = 3e5 * dz_err
    
    good_vel = dv < 5000
    
    if np.sum(good_vel) > 0:
    
        sep_r26 = np.insert(sep_r26[good_vel], 0, 0.)
        dv = np.insert(dv[good_vel], 0, 0.)
        dv_err = np.insert(dv_err[good_vel], 0, 3e5*zc_err)

        # Extract the 0.33xR26 points.
        is_033_r26 = (sep_r26 > 0.35) & (sep_r26 < 0.45)
        
        if np.sum(is_033_r26) > 0:
            v033 = np.mean(dv[is_033_r26]) / np.sqrt(1 - cosi**2)
            dv033 = np.sqrt(np.sum(dv_err[is_033_r26]**2)) / np.sqrt(1 - cosi**2)
            z.append(zc)
            rmag_clean.append(float(sga_galaxy['R_MAG_SB26_1']))
            rmag_err_clean.append(float(sga_galaxy['R_MAG_SB26_ERR_1']))
            vmax_clean.append(v033)
            dvmax_clean.append(dv033)
            abs_mag_clean.append(float(sga_galaxy['R_MAG_SB26_1'] - sga_galaxy['DM1_SN'] - 5*np.log10(0.742)))
            abs_mag_err_clean.append(np.sqrt((float(sga_galaxy['R_MAG_SB26_ERR_1']))**2 + (float(sga_galaxy['e_DM1_SN']))**2))
    
#inComa_sga_table[['SGA_ID', 'BA', 'cosi']].show_in_notebook()
print(len(rmag_clean))

In [ ]:
def l1norm(pars, x, y, dy):
    '''
    Linear fit that uses the l1-norm (robust against outliers).
    '''
    a, b = pars
    return np.sum(np.abs((y - a - b*x)/dy))

def l1norm_noerror(pars, x, y):
    '''
    Linear fit that uses the l1-norm without normalizing by measurement uncertainties.
    '''
    a, b = pars
    return np.sum(np.abs(y - a - b*x))

def l2norm(pars, x, y, dy):
    '''
    Linear fit that uses the l2-norm
    '''
    a, b = pars
    return np.sum((y - a - b*x)**2/dy**2)

def fit_tfr(r, logv, dlogv):
    fmin = 1e99
    a, b = -0.5, -0.15
    hess_inv = np.ones((2,2))
    
    succ_res = None
    
    # Try a large number of random seeds to ensure a decent fit.
    for i in range(1000):
        _a, _b = np.random.uniform(-1,1), np.random.uniform(-0.14306432,-0.14306432)
        
        res = minimize(l1norm_noerror, 
                       [_a, _b], 
                       args=(r,logv),# dlogv),
                       method='L-BFGS-B', 
                       bounds=[[-1,1], [-0.14306432,-0.14306432]])
        
        if res.fun < fmin and res.success:
            # print('Successful fit')
            succ_res = res.copy()
            fmin = res.fun
            a, b = res.x
            hess_inv = res.hess_inv
    
    if succ_res is None:
        print('No successful fits')
    # else:
    #     print(succ_res)
    
    return a, b, hess_inv

In [ ]:
abs_mag_clean = np.asarray(abs_mag_clean)
abs_mag_err_clean = np.asarray(abs_mag_err_clean)
vmax_clean = np.asarray(vmax_clean)
dvmax_clean = np.asarray(dvmax_clean)

log10vmax_clean = np.log10(vmax_clean)
dlog10vmax_clean = 0.434 * dvmax_clean / vmax_clean 

a, b, hess_inv = fit_tfr(abs_mag_clean, 
                  log10vmax_clean, 
                  dlog10vmax_clean)

print('Fitted params: a={0}, b={1}'.format(a, b))
print('Slope={0}, y-int={1}'.format(1/b, -a/b))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6), tight_layout=True)

ax.errorbar(log10vmax_clean, 
            abs_mag_clean,
            yerr = abs_mag_err_clean,
            xerr=dlog10vmax_clean, 
            fmt='ro', 
            alpha=0.7,
            label='Cleaned data, $\cos{(i)}$-corrected')

# a = -0.75655391
# b = -0.14665

logv = np.arange(2, 2.5, 0.03)
ax.plot(logv, 1/b * (logv - a), 'r--', alpha=0.8)
# ################################################################################


ax.set(ylabel='$M_r$(26)-5log$_{10}$(h)',
       #ylim=(-18, -38),
       xlabel=r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$',
        ylim=(-18, -22),
       title=r'Rotational velocity at $0.4R_{26}$')
ax.grid(ls=':')
# ax.legend(loc='upper left', fontsize=10);

# fig.suptitle(r'Max velocity at $0.33\times R_{26}$', y=1.05)
# fig.subplots_adjust(top=0.8)
#fig.savefig('tf_dist_04_06_2022.png', dpi=120, transparent=True)

# print(np.sum(isrmeas))

In [ ]:
# Create a 2x2xN matrix.
ndata = len(dlog10vmax_clean)
cov3 = np.empty((2, 2, ndata))

# loop over arrays of uncertainties in logv and mag
# Assume diagonal covariance for each measurement.
for i, (dlogv, dm) in enumerate(zip(dlog10vmax_clean, abs_mag_err_clean)):
    cov3[:,:,i] = np.array([[dlogv**2, 0.], [0., dm**2]])
                              
# cov[:,:,0]

In [ ]:
logv = log10vmax_clean
mr = abs_mag_clean
logv.shape, mr.shape

In [ ]:
data3 = [logv, mr]
cov3 = cov3

In [ ]:
hf = LinFit([logv, mr], cov3)

In [ ]:
bounds = ((-6.99, -6.98), (-10.0, 10.0), (1.0e-5, 500.0))
mcmc_samples, mcmc_lnlike = hf.emcee(bounds)
print(np.mean(mcmc_samples, axis=1), np.std(mcmc_samples, axis = 1))

In [ ]:
a, b, sig    = np.mean(mcmc_samples, axis=1)
da, db, dsig = np.std(mcmc_samples, axis=1)

for val, err in zip((a, b, sig), (da, db, dsig)):
    print('{:6.2f} +/- {:.2f}'.format(val, err))

In [ ]:
sigmas = hf.get_sigmas()
xvals = np.linspace(0., 3., 1000)
yvals = hf.coords[0] * xvals + hf.coords[1]

# Get the MCMC 1-sigma quantiles to plot with the fit.
y_chain = np.outer(xvals, mcmc_samples[0]) + mcmc_samples[1]
y_chain_quantiles = np.quantile(y_chain, [0.1587, 0.8414], axis=1)

# Pack info into data
data = [log10vmax_clean, abs_mag_clean, ]
x_err = dlog10vmax_clean
y_err = abs_mag_err_clean 
corr_xy = np.zeros_like(x_err)

# Generate ellipses
ells = [
    Ellipse(
        xy=[data[0][i], data[1][i]],
        width=2.0 * y_err[i],
        height=2.0 * x_err[i],
        angle=np.rad2deg(np.arccos(corr_xy[i])),
    )
    for i in range(len(data[0]))
]

# Make the plot
fig = plt.figure(figsize=(12,7))
ax = fig.add_axes([0.15, 0.15, 1.03, 0.83])
for i, e in enumerate(ells):
    ax.add_artist(e)
    e.set_color(cmo.cm.matter(sigmas[i] / np.amax(sigmas)))
    e.set_edgecolor('None')
    e.set_alpha(0.9)
ax.fill_between(xvals, y_chain_quantiles[0], y_chain_quantiles[1], color="k", alpha=0.2)
ax.plot(xvals, yvals, c="k", marker="None", ls="-", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals - hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.plot(xvals, yvals + hf.vert_scat, c="k", marker="None", ls="--", lw=1.3, alpha=0.9)
ax.set_xlabel(r'$\log{(v_\mathrm{max} / \mathrm{km~s}^{-1})}$', fontsize=24)
ax.set_ylabel('$M_r$(26)-5log$_{10}$(h)', fontsize=24)
ax.set_title(r"Independent Distance Measurements", fontsize=24)
ax.set_xlim(1.9, 2.5)
ax.set_ylim(-18, -22)

# Add the colourbar
cb = fig.colorbar(
    cm.ScalarMappable(norm=colors.Normalize(vmin=0.0, vmax=np.amax(sigmas)), cmap = cmo.cm.matter),
    ax=ax,
    shrink=0.5,
    aspect=10,
    anchor=(-6.6, 0.95),
)
cb.set_label(label=r"$\sigma$", fontsize=16)
plt.savefig('inddist.png', bbox_inches='tight')

In [ ]:
plt.hist(yrand, bins = 30)
#plt.xlim(-3,-6)
plt.axvline(yrand.mean(), ymin=-1, ymax=15, color='red', label='mean')
plt.axvline(x=yrand.mean() - sty * 1.96, ymin=-1, ymax=15, color='green', label='95% CI')
plt.axvline(x=yrand.mean() + sty * 1.96, ymin=-1, ymax=15, color='green')
plt.title('Hyperfit MCMC Y-Intercept Values')